In [171]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [180]:
#working with simple data ,(no preprocessing done now),suppose these are the 4 documemnts
docs = ["How to format my hard disk", " Hard disk format problems ", "format hard disk correctly", "I had an apple"]


In [181]:
count_vect = CountVectorizer( stop_words='english')
X_train_counts = count_vect.fit_transform(docs)
(X_train_counts)

<4x6 sparse matrix of type '<type 'numpy.int64'>'
	with 12 stored elements in Compressed Sparse Row format>

In [182]:
count_vect.get_feature_names()

[u'apple', u'correctly', u'disk', u'format', u'hard', u'problems']

In [183]:
tfIdfVect = TfidfVectorizer(analyzer='word',stop_words='english')
tfidf = tfIdfVect.fit_transform(docs)

In [184]:
print (tfidf)

  (0, 3)	0.57735026919
  (0, 4)	0.57735026919
  (0, 2)	0.57735026919
  (1, 3)	0.428175115973
  (1, 4)	0.428175115973
  (1, 2)	0.428175115973
  (1, 5)	0.670819059199
  (2, 3)	0.428175115973
  (2, 4)	0.428175115973
  (2, 2)	0.428175115973
  (2, 1)	0.670819059199
  (3, 0)	1.0


In [185]:
from sklearn.metrics.pairwise import cosine_similarity


In [186]:
#finding cosine similarity with doc1 with other documents
cosine_similarities = cosine_similarity(tfidf[0:1], tfidf)



In [187]:
cosine_similarities

array([[ 1.        ,  0.74162106,  0.74162106,  0.        ]])